In [1]:
import numpy as np
import numpy.fft as fft
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import signal
import sklearn
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torchvision
import torchaudio
from torchvision import transforms
import cv2
import time, sys, os

from utils import MindBigData, GetDataSet, GetDataLoaders, GetDataLoadersEEGImages, GetDataAndPreProcess
from MultilayerBidirectionalRNN import MultilayerBidirectionalRNN

seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline


C:\Users\ryoii\AppData\Roaming\Python\Python310\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
"""
Load EEG data from specified data path, with specified number of samples per digit.
- GetDataSet : standardization, min-max scaling within std 
    x : 振幅 in microVolts
    y : target digit
- Preprocess : butterworth filter(highpass 0.1Hz + 平滑化),
 notch filter(ノイズ除去 50Hz), trim first 32 samples
"""
dataName = "EP/EP1.01.txt"
# dataName = "MUSE/MU.txt"
dataPath = os.path.join(os.path.dirname(os.getcwd()), dataName)

x_raw, x_preprocessed, x_reconstructed, x_standardized, y = GetDataAndPreProcess(input_file=dataPath)


[5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]


In [3]:
# (trial, channel, data points)
print(x_raw.shape)
print(x_preprocessed.shape)
print(x_reconstructed.shape)
print(x_standardized.shape)


(50000, 14, 256)
(50000, 14, 224)
(50000, 14, 224)
(50000, 14, 224)


In [4]:
"""
Prepare dataloaders 75%:15%:10%
"""
train_loader_raw, valid_loader_raw, test_loader_raw = GetDataLoaders(x_raw, y, batch_size=64)
train_loader_preprocessed, valid_loader_preprocessed, test_loader_preprocessed = GetDataLoaders(x_preprocessed, y, batch_size=64)
train_loader_reconstructed, valid_loader_reconstructed, test_loader_reconstructed = GetDataLoaders(x_reconstructed, y, batch_size=64)
train_loader_standardized, valid_loader_standardized, test_loader_standardized = GetDataLoaders(x_standardized, y, batch_size=64)

In [5]:
# Initialize the model
model = MultilayerBidirectionalRNN(input_shape=(14, 256), num_classes=10)

# Train the model and save the best one
model.train(train_loader_standardized, validation_loader=valid_loader_standardized, epochs=10, checkpoint_path='best_model.keras')

# Evaluate the best model on the test set
test_loss, test_acc = model.evaluate(test_loader_standardized)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}')

Epoch 1/10


ValueError: Input 0 of layer "functional" is incompatible with the layer: expected shape=(None, 14, 256), found shape=(None, 14, 224)

In [ ]:
# Display 2 signals with their labels
data_iter = iter(train_loader_raw)
inputs, labels = next(data_iter)
EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

fig = plt.figure(figsize=(20, 15))
for i in range(1):
    for j in range(14):
        ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
        if (j == 0):
            ax.set_title(
                'Raw : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
        ax.plot(inputs[i][j])
        ax.set_xlabel('samples')
        ax.set_ylabel(EEGChannel[j])
        ax.grid()

In [ ]:
# Display 2 signals with their labels
data_iter = iter(train_loader_preprocessed)
inputs, labels = next(data_iter)
EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

fig = plt.figure(figsize=(20, 15))
for i in range(1):
    for j in range(14):
        ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
        if (j == 0):
            ax.set_title(
                'Preprocessed : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
        ax.plot(inputs[i][j])
        ax.set_xlabel('samples')
        ax.set_ylabel(EEGChannel[j])
        ax.grid()

In [ ]:
# Display 2 signals with their labels
data_iter = iter(train_loader_reconstructed)
inputs, labels = next(data_iter)
EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

fig = plt.figure(figsize=(20, 15))
for i in range(1):
    for j in range(14):
        ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
        if (j == 0):
            ax.set_title(
                'reconstructed : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
        ax.plot(inputs[i][j])
        ax.set_xlabel('samples')
        ax.set_ylabel(EEGChannel[j])
        ax.grid()

In [ ]:
# Display 2 signals with their labels
data_iter = iter(train_loader_standardized)
inputs, labels = next(data_iter)
EEGChannel = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]

fig = plt.figure(figsize=(20, 15))
for i in range(1):
    for j in range(14):
        ax = fig.add_subplot(14, 2, i + 1 + 2 * j)
        if (j == 0):
            ax.set_title(
                'standardized : digit seen by patient: ' + str(labels[i].item()))  # .item() gets the value contained in a tensor
        ax.plot(inputs[i][j])
        ax.set_xlabel('samples')
        ax.set_ylabel(EEGChannel[j])
        ax.grid()